# What is Machine Learning?

Machine learning discovers patterns within data without being explicitly programmed.  This lesson introduces machine learning, explores the main topics in the field, and builds an end-to-end pipeline in Spark.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
* Define machine learning
* Differentiate supervised and unsupervised tasks
* Identify regression and classification tasks
* Train a model, interpret the results, and create predictions

<iframe  
src="//fast.wistia.net/embed/iframe/h8cqzugdrf?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/h8cqzugdrf?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

### Learning from Data

Machine learning refers to a diverse set of tools for understanding data.  More technically, **machine learning is the process of _learning from data_ without being _explicitly programmed_**.  Let's unpack what that means.

Take a dataset of Boston home values in the 1970's for example.  The dataset consists of the value of homes as well as the number of rooms, crime per capita, and percent of the population considered lower class.  Home value is the _output variable_, also known as the _label_.  The other variables are known as _input variables_ or _features_.

A machine learning model would _learn_ the relationship between housing price and the various features without being explicitly programmed.  There are many possible functions that would map input features to the output variable.  In more technical terms, our model would learn a function `f()` that maps the relationship between input features and the output variable.

The following image shows the relation between our features and house value.  A good model `f()` would learn from the data that the number of rooms in a home is positively correlated to the house value while crime and percent of the neighborhood that is lower class is negatively correlated.  

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/boston-housing.png" style="height: 400px; margin: 20px"/></div>

The lines above represent the best fit for the data where our model's best guess for a house price given a feature value on the X axis is the corresponding point on the line.

**Machine learning is the set of approaches for estimating this function `f()` that maps features to an output.**  The inputs to this function can range from stock prices and customer information to images and DNA sequences.  Many of the same statistical techniques apply regardless of the domain.  This makes machine learning a generalizable skill set that drives decision-making in modern businesses.

### Getting Started

Run the following cell to configure our "classroom."

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Remember to attach your notebook to a cluster. Click <b>Detached</b> in the upper left hand corner and then select your preferred cluster.

<img src="https://s3-us-west-2.amazonaws.com/curriculum-release/images/eLearning/attach-to-cluster.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtrainwesteurope.blob.core.windows.net/

Created user-specific database

Using the database saifahmed_k_outlook_com_db .

All done!

### Supervised vs Unsupervised Learning

Machine learning problems are roughly categorized into two main types:<br><br>

* **Supervised learning** looks to predict the value of some outcome based on one or more input measures
  - Our example of the Boston Housing Dataset is an example of supervised learning
  - In this case, the output is the price of a home and the input is features such as number of rooms
* **Unsupervised learning** describes associations and patterns in data without a known outcome
  - An example of this would be clustering customer data to find the naturally occurring customer segments
  - In this case, no known output is used as an input.  Instead, the goal is to discover how the data are organized into natural segments or clusters

This course will cover supervised learning, which is the vast majority of machine learning use cases in industry.  Later courses will look at unsupervised approaches.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/regression.png" style="height: 400px; margin: 20px"/><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/clustering.png" style="height: 400px; margin: 20px"/></div>

Import the Boston dataset, which contains median house values in 1000's (`medv`) for a variety of different features.  Since this dataset is "supervised" my the median value, this is a supervised machine learning use case.

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/training/"))


In [0]:
bostonDF = (spark.read
  .option("HEADER", True)
  .option("inferSchema", True)
  .csv("/mnt/training/bostonhousing/bostonhousing/bostonhousing.csv")
)

display(bostonDF)

Spark differs from many other machine learning frameworks in that we train our model on a single column that contains a vector of all of our features.  Prepare the data by creating one column named `features` that has the average number of rooms, crime rate, and poverty percentage.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.feature.VectorAssembler" target="_blank">`VectorAssembler` documentation for more details.</a>
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> `VectorAssembler` is a tranformer, which implements a `.transform()` method.  Estimators, by contrast, need to learn from the data using a `.fit()` method before they can transform data.

In [0]:
from pyspark.ml.feature import VectorAssembler

featureCols = ["rm", "crim", "lstat"]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

bostonFeaturizedDF = assembler.transform(bostonDF)

display(bostonFeaturizedDF)

```
from pyspark.ml.feature import VectorAssembler

featureCols = ["rm", "crim", "lstat"]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
```
### Why Do We Need to Put Them in One Vector?

Most machine learning algorithms in Spark (and other frameworks) expect the features to be in the form of a **single vector**. A vector is just a list of numbers, and it's easier for models to process this than handling many separate columns. By combining these columns into one vector, the algorithm can efficiently calculate predictions.

Imagine you are predicting house prices, and you have three features:
- `rm`: Average number of rooms.
- `crim`: Crime rate.
- `lstat`: Percentage of lower-income households.

For example, let’s say you have a row in the dataset with the following values:
- `rm = 6.0` (6 rooms)
- `crim = 0.1` (low crime rate)
- `lstat = 12.0` (12% lower-income population)

Instead of keeping these values in separate columns, we combine them into a **feature vector** like this:
```
features = [6.0, 0.1, 12.0]
```

#### Why Combine Them?
- **Efficiency**: Machine learning algorithms need to process many features at once. It's faster and more efficient to work with a single vector (list of numbers) than with multiple columns.
  
- **Mathematical Operations**: The model often needs to perform operations like matrix multiplication or dot products on the features. These operations are easier when all the features are stored in one vector.
  
- **Example of How the Model Uses the Vector**: 
   Suppose we are training a **linear regression model**. The model might learn an equation like this to predict house prices:
   ```
   predicted_price = (5.2 * rm) - (0.1 * crim) - (0.6 * lstat) - 2.6
   ```
   To make this prediction for the above row, the model will compute:
   ```
   predicted_price = (5.2 * 6.0) - (0.1 * 0.1) - (0.6 * 12.0) - 2.6
                   = 31.2 - 0.01 - 7.2 - 2.6
                   = 21.39
   ```

In this case, having all the features in a vector `[6.0, 0.1, 12.0]` makes it easy for the model to apply these calculations.

- We need to combine them into one vector because machine learning algorithms work more efficiently with data in this format, allowing them to process many features together and apply mathematical operations for training and making predictions.

The line of code:

```python
bostonFeaturizedDF = assembler.transform(bostonDF)
```

is performing a **data transformation** step using the `VectorAssembler` object (`assembler`). Here's what it does in detail:

### Explanation

1. **Input: `bostonDF` (Original DataFrame)**:
   - `bostonDF` is the original Spark DataFrame containing the Boston Housing dataset. It has multiple columns, including features like `rm` (average number of rooms), `crim` (crime rate), and `lstat` (percentage of lower-income population), as well as the target variable (`medv`), which is the median house value.

2. **`assembler.transform()`**:
   - The `assembler` is an instance of `VectorAssembler`, which was created earlier with a list of input columns (`["rm", "crim", "lstat"]`) and an output column (`"features"`).
   - The `.transform(bostonDF)` method is applied to the original DataFrame (`bostonDF`). It combines the specified input columns (`rm`, `crim`, `lstat`) into a single **feature vector** column, called `"features"`.

3. **Output: `bostonFeaturizedDF` (Transformed DataFrame)**:
   - After applying the transformation, a new DataFrame (`bostonFeaturizedDF`) is created. This DataFrame contains all the original columns, plus a new column called `features`, which holds the combined feature vectors for each row.
   
   - For example, for a row with:
     - `rm = 6.0` (6 rooms),
     - `crim = 0.1` (crime rate),
     - `lstat = 12.0` (12% lower-income population),
     
     the new `features` column would contain:
     ```
     features = [6.0, 0.1, 12.0]
     ```

### Why It's Done

- **Combining Features for Machine Learning**: Machine learning algorithms in Spark typically expect input features to be in a single column (a vector of numbers). By combining multiple feature columns into one vector column (`features`), the model can use this as input for training.
- **Simplifies Data Processing**: Instead of passing multiple columns separately, it's easier and more efficient to pass one column that contains all the features.

### Example Output Structure
If `bostonDF` originally looked like this:

| rm  | crim  | lstat | medv  |
|-----|-------|-------|-------|
| 6.0 | 0.1   | 12.0  | 21.2  |
| 7.0 | 0.2   | 15.0  | 25.3  |

After transformation with `VectorAssembler`, `bostonFeaturizedDF` will look like this:

| rm  | crim  | lstat | medv  | features       |
|-----|-------|-------|-------|----------------|
| 6.0 | 0.1   | 12.0  | 21.2  | [6.0, 0.1, 12.0] |
| 7.0 | 0.2   | 15.0  | 25.3  | [7.0, 0.2, 15.0] |

Now the `features` column contains a vector of the selected features (`rm`, `crim`, `lstat`), which can be fed into a machine learning model for training.

### Summary:
The line `bostonFeaturizedDF = assembler.transform(bostonDF)` takes the original DataFrame (`bostonDF`) and combines the selected feature columns (`rm`, `crim`, `lstat`) into a single `features` column, creating a new DataFrame (`bostonFeaturizedDF`) that is ready for machine learning algorithms.

We now have the input features and the output, median value, which appears in the data as `medv`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See <a href="https://spark.apache.org/docs/latest/mllib-data-types.html" target="_blank">the MLlib documentation for more details</a> on the sparse vector notation seen in the `features` column, which will be covered in more detail in the Featurization lesson.

### Regression vs Classification

Variables can either be quantitative or qualitative:<br><br>

* **Quantitative** values are numeric and generally unbounded, taking any positive or negative value
* **Qualitative** values take on a set number of classes or categories

| Variable type    | Also known as         | Examples                                                          |
|:-----------------|:----------------------|:------------------------------------------------------------------|
| quantitative     | continuous, numerical | age, salary, temperature                                          |
| qualitative      | categorical, discrete | gender, whether or a not a patient has cancer, state of residence |

Machine learning models operate on numbers so a qualitative variable like gender, for instance, would need to be encoded as `0` for male or `1` for female.  In this case, female isn't "one more" than male, so this variable is handled differently compared to a quantitative variable.

Generally speaking, **a supervised model learning a quantitative variable is called regression and a model learning a qualitative variable is called classification.**

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/classification_v_regression.jpg" style="height: 400px; margin: 20px"/>

Import linear regression, one way of modeling continuous variables.  Set the output to be the `medv` variable and the input to be the `features` made above.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.regression.LinearRegression" target="_blank">LinearRegression documentation for more details.</a>

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="medv", featuresCol="features")

Fit the model to the data using the `.fit()` method.  This returns a trained model that has learned from our data and will be stored in `lrModel`

In [0]:
lrModel = lr.fit(bostonFeaturizedDF)

Take a look at what the model learned from our data.  In the case of linear regression, this comes in the form of an equation that describes the relationship the model has just learned.  This will be covered in more detail in later lessons.

In [0]:
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))

The code:

```python
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
```

is displaying the **coefficients** and **intercept** of a **linear regression model** that has been trained. Here's what each part of this code does:

### 1. **Displaying the Coefficients**

```python
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
```

- **What are coefficients?**
  - Coefficients represent the weight or influence each feature has on the prediction made by the linear regression model. Each coefficient corresponds to one of the input features. They show how much the target variable (e.g., house price) changes when the corresponding feature changes, assuming the other features remain constant.

- **Breakdown of the code:**
  - `lrModel.coefficients`: This retrieves the coefficients from the trained linear regression model (`lrModel`). In this case, `lrModel.coefficients` would be a vector (list of numbers) containing three values because the model is using three features (`rm`, `crim`, `lstat`).
  - `*lrModel.coefficients`: The `*` unpacks the list of coefficients into separate arguments for the `format()` function.
  - `"{0:.1f}, {1:.1f}, {2:.1f}".format(...)`: The `.format()` method inserts the coefficients into the string. The `{:.1f}` specifies that each number will be formatted to one decimal place.
  
- **Example**: If the coefficients are `[5.2, -0.1, -0.6]`, the output will be:
  ```
  Coefficients: 5.2, -0.1, -0.6
  ```
  This means:
  - For every additional room (`rm`), the predicted house price increases by 5.2 (assuming the other features are held constant).
  - For every 1-point increase in the crime rate (`crim`), the predicted house price decreases by 0.1.
  - For every 1% increase in the lower-income population (`lstat`), the predicted house price decreases by 0.6.

### 2. **Displaying the Intercept**

```python
print("Intercept: {0:.1f}".format(lrModel.intercept))
```

- **What is the intercept?**
  - The intercept is the baseline value of the target variable (in this case, the house price) when all the features are equal to zero. It’s the starting point of the prediction when none of the input features have any effect (i.e., they are zero).

- **Breakdown of the code:**
  - `lrModel.intercept`: This retrieves the intercept value from the trained model.
  - `"{0:.1f}".format(...)`: This formats the intercept to one decimal place and inserts it into the string.

- **Example**: If the intercept is `-2.6`, the output will be:
  ```
  Intercept: -2.6
  ```
  This means that if the values for `rm`, `crim`, and `lstat` were all zero, the predicted house price would start at -2.6 (which is a hypothetical value since these features are unlikely to be exactly zero in real life).

### Why Are Coefficients and Intercept Important?

In a **linear regression** model, the equation for making predictions is:
```
predicted_value = (coefficient_1 * feature_1) + (coefficient_2 * feature_2) + ... + intercept
```

For this example:
```
predicted_price = (5.2 * rm) + (-0.1 * crim) + (-0.6 * lstat) - 2.6
```

These coefficients and the intercept tell you how the model is calculating its predictions based on the input features.

### Example Output:

If your model has learned the following:
- Coefficients: `5.2`, `-0.1`, `-0.6`
- Intercept: `-2.6`

The output would be:
```
Coefficients: 5.2, -0.1, -0.6
Intercept: -2.6
```

This tells you that:
- The house price increases by 5.2 for each additional room.
- The house price decreases by 0.1 for each increase in the crime rate.
- The house price decreases by 0.6 for each 1% increase in the lower-income population.
- The starting point (intercept) is -2.6.

### Summary:
- The first `print()` line displays the coefficients (weights) for each feature (`rm`, `crim`, and `lstat`), showing how they influence the predicted house price.
- The second `print()` line displays the intercept, which is the base value when all features are zero.


The coefficients of our model are a multiplier against our original features and the intercept is a constant.  To interpret our model, use the following equation:

&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = (5.2 x number of rooms) - (.1 x crime rate) - (.6 x % lower class) - 2.6`

## Prediction

So far, we've accomplished the following:<br><br>

* Prepared data for a machine learning model
* Trained a linear regression model
* Interpreted what it learned about our data

Now, let's see how it predicts on data it has already seen as well as new data.

Create a subset of the first 10 rows of the `features` and `medv` target variable.

In [0]:
subsetDF = (bostonFeaturizedDF
  .limit(10)
  .select("features", "medv")
)

display(subsetDF)

Use the `transform` method on the trained model to see its prediction.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Now that `lrModel` is a trained estimator, we can transform data using its `.transform()` method.

In [0]:
predictionDF = lrModel.transform(subsetDF)

display(predictionDF)

How did the model do?  While there's a difference between the prediction and the true value (also known as the model error), generally the model seems to have learned something about our data.

Now predict off of a hypothetical data point of a 6 bedroom home with a 3.6 crime rate and 12 % average lower class.  According to our formula, the model should predict about 21:

&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = (5.2 x number of rooms) - (.1 x crime rate) - (.6 x % lower class) - 2.6`<br>
&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = (5.2 x 6) - (.1 x 3.6) - (.6 x 12) - 2.6`<br>
&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = 31.2 - .4 - 7.2 - 2.6`<br>
&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = 21`<br>

In [0]:
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([6., 3.6, 12.]), )]              # Creates our hypothetical data point
predictDF = spark.createDataFrame(data, ["features"])

display(lrModel.transform(predictDF))